In [34]:
from pydantic import BaseModel
from cerebras.cloud.sdk import Cerebras
from dotenv import load_dotenv
load_dotenv()
import os
import json

cerebras_api_key = os.getenv("CEREBRAS_API_KEY", "")
client = Cerebras(api_key=cerebras_api_key)

class Movie(BaseModel):
    title: str
    director: str
    year: int


class Cinema(BaseModel):
    movies: List[Movie] = []

movie_schema = Cinema.model_json_schema()
movie_schema


{'$defs': {'Movie': {'properties': {'title': {'title': 'Title',
     'type': 'string'},
    'director': {'title': 'Director', 'type': 'string'},
    'year': {'title': 'Year', 'type': 'integer'}},
   'required': ['title', 'director', 'year'],
   'title': 'Movie',
   'type': 'object'}},
 'properties': {'movies': {'default': [],
   'items': {'$ref': '#/$defs/Movie'},
   'title': 'Movies',
   'type': 'array'}},
 'title': 'Cinema',
 'type': 'object'}

In [35]:
completion = client.chat.completions.create(
    model="llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that generates movie recommendations.",
        },
        {
            "role": "user",
            "content": "Suggest a sci-fi movie from the 1990s and a comedy",
        },
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "movie_schema",
            "strict": True,
            "schema": movie_schema,
        },
    },
)

movie_data = json.loads(completion.choices[0].message.content)
print(json.dumps(movie_data, indent=2))

{
  "movies": [
    {
      "title": "The Matrix",
      "director": "The Wachowskis",
      "year": 1999
    },
    {
      "title": "Groundhog Day",
      "director": "Harold Ramis",
      "year": 1993
    }
  ]
}


In [36]:
from typing import Dict, List, Optional, Union, Literal
from pydantic import BaseModel, Field
import uuid
from enum import Enum


class NodeStatus(str, Enum):
    pending = "pending"
    in_progress = "in_progress"
    completed = "completed"
    failed = "failed"


class DependencyType(str, Enum):
    hard = "hard"
    soft = "soft"


class Edge(BaseModel):
    source: str
    target: str
    dependency_type: DependencyType = DependencyType.hard
    data_transfer: Optional[Dict] = None


class Node(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    description: str
    status: NodeStatus = NodeStatus.pending
    inputs: Optional[Dict] = None
    outputs: Optional[Dict] = None


class TaskGraph(BaseModel):
    nodes: Dict[str, Node] = {}
    edges: List[Edge] = []

In [37]:
# from graph import TaskGraph, Node, Edge

graph_schema = TaskGraph.model_json_schema()
graph_schema

{'$defs': {'DependencyType': {'enum': ['hard', 'soft'],
   'title': 'DependencyType',
   'type': 'string'},
  'Edge': {'properties': {'source': {'title': 'Source', 'type': 'string'},
    'target': {'title': 'Target', 'type': 'string'},
    'dependency_type': {'$ref': '#/$defs/DependencyType', 'default': 'hard'},
    'data_transfer': {'anyOf': [{'additionalProperties': True,
       'type': 'object'},
      {'type': 'null'}],
     'default': None,
     'title': 'Data Transfer'}},
   'required': ['source', 'target'],
   'title': 'Edge',
   'type': 'object'},
  'Node': {'properties': {'id': {'title': 'Id', 'type': 'string'},
    'description': {'title': 'Description', 'type': 'string'},
    'status': {'$ref': '#/$defs/NodeStatus', 'default': 'pending'},
    'inputs': {'anyOf': [{'additionalProperties': True, 'type': 'object'},
      {'type': 'null'}],
     'default': None,
     'title': 'Inputs'},
    'outputs': {'anyOf': [{'additionalProperties': True, 'type': 'object'},
      {'type': 'n

In [38]:
completion = client.chat.completions.create(
    model="llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that generates graph plan recommendations.",
        },
        {
            "role": "user",
            "content": "Bake a cake",
        },
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "graph_schema",
            "strict": True,
            "schema": graph_schema,
        },
    },
)
graph_data = json.loads(completion.choices[0].message.content)
print(json.dumps(graph_data, indent=2))

BadRequestError: Error code: 400 - {'message': "Object fields require at least one of: 'properties' or 'anyOf' with a list of possible properties.", 'type': 'invalid_request_error', 'param': 'response_format', 'code': 'wrong_api_format'}